In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
import mysql.connector
spark=SparkSession.builder.master('local[1]').appName('Credit Card Management System').getOrCreate()

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="admin",
  database="creditcard_capstone"
)
mycursor = mydb.cursor()

In [19]:
def spark_read(table):
    temp=spark.read\
            .format("jdbc")\
            .option("driver","com.mysql.cj.jdbc.Driver")\
            .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone")\
            .option("dbtable", table)\
            .option("user", "root")\
            .option("password", "admin")\
            .load()
    return temp

In [20]:
# Customer account details
df_cust=spark_read("cdw_sapp_customer")
df_cust.createOrReplaceTempView('df_cust')
# Branch details
df_branch = spark_read("cdw_sapp_branch")
df_branch.createOrReplaceTempView('df_branch')
#credit card details
df_credit = spark_read("cdw_sapp_credit_card")
df_credit.createOrReplaceTempView('df_credit')

## TRANSACTION DETAILS
1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.

In [10]:
#and last_updated='2018-04-21 12:49:02' order by EXTRACT(DAY FROM last_updated) desc
def customer_transactions():
    try:
        zipcode=int(input('Enter the zipcode of location you want to analyze: '))
        month=int(input('Enter month number: '))
        year=int(input('Enter year'))
        query=f"select * from df_credit join df_cust on df_cust.ssn=df_credit.cust_ssn where df_cust.cust_zip={zipcode} \
        and month(timeid)={month} and year(timeid)= {year} order by day(timeid) desc"
        df_trans=spark.sql(query)
        df_trans.show()
    except Exception as e:
        print("Please try to enter valid details\n",e)


2)    Used to display the number and total values of transactions for a given type.

In [11]:
#spark.sql("select count(*),round(sum(transaction_value),2) as total_value from df_credit where transaction_type='Education'").show()

def transaction_value():
    try:
        type=input('Enter the type of transaction: ').title()
        query=f"select count(*),round(sum(transaction_value),2) as total_value from df_credit where transaction_type='{type}'"
        df_value=spark.sql(query)
        df_value.show()
    except Exception as e:
        print('Please enter a valid Transaction Type')

3)    Used to display the total number and total values of transactions for branches in a given state.

In [12]:
def branch_transactions():
    try:
        state=input("Enter a State code to find its transaction details: ").upper()
        query=f"select count(*),round(sum(transaction_value),2) as total_Value from df_credit join df_branch \
        on df_credit.branch_code= df_branch.branch_code where branch_state='{state}'"
        spark.sql(query).show()
    except Exception as e:
        print("Please enter valid State code")

## CUSTOMER DETAILS
4) Used to check the existing account details of a customer.

In [ ]:
def cust_acct_details():
    ssn=int(input("Enter customer SSN: "))
    query=f"select * from df_credit join df_cust on df_cust.ssn=df_credit.cust_ssn \
    join df_branch on df_credit.branch_code=df_branch.branch_code\
    where df_cust.ssn={ssn}"
    spark.sql(query).show()

5) Used to modify the existing account details of a customer.

In [ ]:
def modify_acct():
    ssn=int(input("Enter customer SSN: "))
    column=input("Enter column name which you want to update: ")
    new_value=input("Enter value you want to update: ")
    query = f"UPDATE cdw_sapp_customer SET {column} = '{new_value}' WHERE ssn = {ssn}"
    mycursor.execute(query)
    mydb.commit()
    print(mycursor.rowcount, "record(s) affected")
    df_cust.createOrReplaceTempView('df_cust')
    query=f'select * from df_cust where ssn={ssn}'
    spark.sql(query).show()

6) Used to generate a monthly bill for a credit card number for a given month and year.

In [ ]:
def ccno_monthly_bill():
    ccno=input("Enter the credit card number to generate monthly bill: ")
    mnth=int(input("Enter the Month: "))
    yr=int(input("Enter year"))
    query=f"select SUM(Transaction_value) as Montly_Bill from df_credit where MONTH(TIMEID)={mnth} and YEAR(TIMEID)={yr} and credit_card_no={ccno}"
    spark.sql(query).show()

7) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order

In [ ]:
def trans_btw_dates():
    ssn=input("Enter the customer ssn number to generate monthly bill: ")
    start_date=datetime.strptime(input("Enter the start date: "),'%Y-%m-%d').date()
    end_date=datetime.strptime(input("Enter the end date: "),'%Y-%m-%d').date()
    query=f"select * from df_credit where cust_ssn = '{ssn}' and timeid >= '{start_date}' and timeid <= '{end_date}' \
    order by year(timeid) desc, month(timeid) desc, day(timeid) desc"
    spark.sql(query).show()

In [44]:
while True:
    id=int(input('''What do you want to analyze?
            1. Display transactions made by customers in a given zipcode for a given month and year.
            2. count and total values based on transaction type
            3. count and total values of transactions for branches in given state
            4. Check the Account Details of a customer
            5. To modify existing account details of a customer
            6. To generate monthly bill for a credit card number for a given month and year
            7. Display the transactions made by a customer between two dates. Order by year, month, and day in descending order
          '''))
    if id==1:
        customer_transactions()
        break
    elif id==2:
        transaction_value()
        break
    elif id==3:
        branch_transactions()
        break
    elif id==4:
        cust_acct_details()
        break
    elif id==5:
        modify_acct()
        break
    elif id==6:
        ccno_monthly_bill()
        break
    elif id==7:
        trans_btw_dates()
        break
    else:
        print("choose between 1 and 7")